In [ ]:
from cmr import CollectionQuery, GranuleQuery, ToolQuery, ServiceQuery, VariableQuery
# requirements: pip install python-dotenv earthaccess
import os
from dotenv import load_dotenv
import earthaccess as ea
import os
import numpy as np
from dotenv import load_dotenv
import earthaccess as ea
import xarray as xr
from dask.diagnostics import ProgressBar

In [2]:

api = CollectionQuery()
collections = api.archive_center("LP DAAC").keyword("AST_L1*").get(5)

print("Collections:")
for collection in collections:
  print(collection["short_name"])

api = GranuleQuery()
granules = api.short_name("OMNO2d").point(-112.73, 42.5).get(3)

print("Granule Titles:")
for granule in granules:
  print(granule["title"])

Collections:
AST_L1T
AST_L1A
AST_L1T
AST_L1B
AST_L1B
Granule Titles:
OMNO2d.003:OMI-Aura_L3-OMNO2d_2004m1001_v003-2019m1121t082956.he5
OMNO2d.003:OMI-Aura_L3-OMNO2d_2004m1002_v003-2019m1120t150658.he5
OMNO2d.003:OMI-Aura_L3-OMNO2d_2004m1003_v003-2019m1121t083143.he5


In [ ]:
# 1. read .env into the current process
load_dotenv()                    # looks for .env in the working directory

# 2. put the values into real environment variables (optional but tidy)
#    (load_dotenv already sets os.environ, so this is just to show the names)
username = os.environ["EARTHDATA_USERNAME"]
password = os.environ["EARTHDATA_PASSWORD"]

# 3. log in – 'environment' tells earthaccess to grab those vars
ea.login(strategy="environment")


/home/ellab/miniconda3/envs/airpol/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#run below steps to download all files
# requirements:
#   pip install earthaccess xarray[complete] python-dotenv h5netcdf fsspec dask numpy


# ── 0. credentials ───────────────────────────────────────────────────────────
load_dotenv()        # reads EARTHDATA_USERNAME / EARTHDATA_PASSWORD
ea.login()           # environment-based login


In [140]:

# ── 1. CMR search for 365 days Jan 2005 OMNO2d granules ───────────────────────────
granules = ea.search_data(
    short_name="OMNO2d",
    version="003",
    temporal=("2005-01-01", "2005-12-31")
)
fobjs = ea.open(granules)     # HTTP file-like handles (streaming)


QUEUEING TASKS | : 100%|██████████| 365/365 [00:00<00:00, 8841.23it/s]
PROCESSING TASKS | : 100%|██████████| 365/365 [00:43<00:00,  8.33it/s]
COLLECTING RESULTS | : 100%|██████████| 365/365 [00:00<00:00, 79060.16it/s]


In [ ]:
#bottleneck
# ── 2. open science fields (NO₂) ─────────────────────────────────────────────
DATA_GRP = "/HDFEOS/GRIDS/ColumnAmountNO2/Data Fields"

# 1. Download and save to disk (will skip already-downloaded files)     ??
files = ea.download(granules, local_path="OMNO2d_2005_raw")

ds = xr.open_mfdataset(
    fobjs,
    engine="h5netcdf",
    group=DATA_GRP,
    combine="nested",
    concat_dim="time",
    chunks={"time": 1},          # keeps RAM tiny
)

QUEUEING TASKS | :   0%|          | 0/365 [00:00<?, ?it/s]

QUEUEING TASKS | : 100%|██████████| 365/365 [00:00<00:00, 7535.21it/s]
PROCESSING TASKS | : 100%|██████████| 365/365 [15:04<00:00,  2.48s/it]
COLLECTING RESULTS | : 100%|██████████| 365/365 [00:00<00:00, 113789.28it/s]
/tmp/ipykernel_2090/2409888290.py:8: UserWarning: The 'phony_dims' kwarg now defaults to 'access'. Previously 'phony_dims=None' would raise an error. For full netcdf equivalence please use phony_dims='sort'.
  ds = xr.open_mfdataset(
/tmp/ipykernel_2090/2409888290.py:8: UserWarning: The 'phony_dims' kwarg now defaults to 'access'. Previously 'phony_dims=None' would raise an error. For full netcdf equivalence please use phony_dims='sort'.
  ds = xr.open_mfdataset(
/tmp/ipykernel_2090/2409888290.py:8: UserWarning: The 'phony_dims' kwarg now defaults to 'access'. Previously 'phony_dims=None' would raise an error. For full netcdf equivalence please use phony_dims='sort'.
  ds = xr.open_mfdataset(
/tmp/ipykernel_2090/2409888290.py:8: UserWarning: The 'phony_dims' kwarg now de

KeyboardInterrupt: 

In [7]:

# ── 3. build explicit lat/lon coordinates for the 0.25° global grid ──────────
# OMNO2d V003 grid: 720 rows (89.875 → -89.875) and 1440 cols (-179.875 → 179.875)
lat = np.arange( 89.875, -90.0,  -0.25)   # N → S  (length 720)
lon = np.arange(-179.875, 180.0,  0.25)   # W → E  (length 1440)

ds = (ds
      .assign_coords({"phony_dim_0": ("phony_dim_0", lat),
                      "phony_dim_1": ("phony_dim_1", lon)})
      .rename({"phony_dim_0": "latitude",
               "phony_dim_1": "longitude"}))

NameError: name 'ds' is not defined

In [ ]:
# ── 4. crop to Northeast U.S. box ────────────────────────────────────────────
BOX = dict(w=-81, e=-73, n=41.5, s=38)   # West, East, North, South
ne_no2 = ds["ColumnAmountNO2TropCloudScreened"].sel(
    latitude=slice(BOX["n"], BOX["s"]),
    longitude=slice(BOX["w"], BOX["e"])
)

In [75]:
# temporary code bc bottleneck block broke-- filter raw 2005 data
from pathlib import Path
import os
import glob
import h5py
import pandas as pd

path = "/home/ellab/air_pollution/src/OMNO2d_2005_raw"

no2_2005 = []

def create_coords(ds):
    # ── 3. build explicit lat/lon coordinates for the 0.25° global grid ──────────
    # OMNO2d V003 grid: 720 rows (89.875 → -89.875) and 1440 cols (-179.875 → 179.875)
    lat = np.arange( 89.875, -90.0,  -0.25)   # N → S  (length 720)
    lon = np.arange(-179.875, 180.0,  0.25)   # W → E  (length 1440)
        
    # Your region of interest
    lat_min, lat_max = 38.0, 41.5
    lon_min, lon_max = -81.0, -73.0

    # Find the index ranges for that region
    lat_inds = np.where((lat >= lat_min) & (lat <= lat_max))[0]
    lon_inds = np.where((lon >= lon_min) & (lon <= lon_max))[0]

    # Use slicing to get a subarray from your data
    lat_start, lat_end = lat_inds[0], lat_inds[-1] + 1  # add +1 to include endpoint
    lon_start, lon_end = lon_inds[0], lon_inds[-1] + 1

    # Now slice the 2D array
    subarray = ds[lat_start:lat_end, lon_start:lon_end]
    return(subarray)


data_directory = '/home/ellab/air_pollution/src/OMNO2d_2005_raw'
no2_pattern = os.path.join(data_directory, '*.he5')
no2_files2005 = sorted(glob.glob(no2_pattern))

for file_name in no2_files2005:   
    f = h5py.File(file_name, 'r')
    group = f['HDFEOS/GRIDS/ColumnAmountNO2/Data Fields']
    data = group['ColumnAmountNO2TropCloudScreened'][()]

    ds_coords = create_coords(data)
    no2_2005.append(ds_coords)
    #new_data = filter_no2(ds_coords)
    #no2_2005.append(new_data)

print(len(no2_2005))
print(no2_2005)


365
[array([[-1.26765060e+30, -1.26765060e+30, -1.26765060e+30,
        -1.26765060e+30, -1.26765060e+30, -1.26765060e+30,
        -1.26765060e+30, -1.26765060e+30, -1.26765060e+30,
         3.32496739e+14,  2.14991383e+14,  2.47788189e+14,
         4.63428850e+14,  4.05704020e+14, -6.58816316e+13,
        -9.47888377e+13,  7.23901554e+13, -3.98406300e+14,
        -6.64601360e+14, -4.47956298e+14,  2.86552521e+13,
        -3.11833500e+13, -2.94448932e+14, -1.44991486e+14,
        -3.64274698e+14,  2.62587908e+14, -4.63066597e+14,
        -3.13401415e+14,  2.73233387e+14,  4.73842602e+14,
         1.69947051e+15,  5.13394956e+14],
       [ 3.30656345e+14, -6.74685675e+14, -7.94753499e+14,
         2.73445233e+14,  1.74978394e+14, -1.80821177e+14,
         2.29242688e+14,  3.69275416e+13, -1.26765060e+30,
        -1.26765060e+30,  1.91506753e+14,  1.15658310e+14,
         1.06672081e+14,  3.63899870e+13, -4.15777832e+14,
        -1.62388486e+14, -2.66295824e+14, -4.22456741e+14,
        

In [78]:
# Step 1: Build full lat/lon arrays for the global grid
lat_full = np.arange(89.875, -90.0, -0.25)     # shape (720,)
lon_full = np.arange(-179.875, 180.0, 0.25)    # shape (1440,)

# Step 2: Determine which lat/lon indices match your region
lat_min, lat_max = 38.0, 41.5
lon_min, lon_max = -81.0, -73.0

lat_inds = np.where((lat_full >= lat_min) & (lat_full <= lat_max))[0]
lon_inds = np.where((lon_full >= lon_min) & (lon_full <= lon_max))[0]

# Step 3: Extract the corresponding lat/lon slices (same length as your data dims)
lat_region = lat_full[lat_inds]
lon_region = lon_full[lon_inds]


#download 2005 filtered data list to one .nc
# Stack into shape (365, 14, 32)
data_stack = np.stack(no2_2005, axis=0)  # axis=0 becomes 'time'

time = pd.date_range("2005-01-01", periods=365, freq="D")

# Create xarray DataArray
da = xr.DataArray(
    data_stack,
    dims=["time", "lat", "lon"],
    coords={"time": time, "lat": lat_region, "lon": lon_region},
    name="NO2TropCloudScreened"
)

# Wrap in a Dataset (optional but cleaner)
ds = xr.Dataset({"NO2TropCloudScreened": da})

ds.to_netcdf("no2_2005_filtered2.nc")


In [53]:
import h5py

f = h5py.File(file_name, 'r')

#Get the HDF5 group; key needs to be a group name from above
group = f['HDFEOS/GRIDS/ColumnAmountNO2/Data Fields']

#Checkout what keys are inside that group.
for key in group.keys():
    print(key)

# This assumes group[some_key_inside_the_group] is a dataset, 
# and returns a np.array:
data = group['ColumnAmountNO2TropCloudScreened'][()]
#Do whatever you want with data

#After you are done
#f.close()

print(data)



ColumnAmountNO2
ColumnAmountNO2CloudScreened
ColumnAmountNO2Trop
ColumnAmountNO2TropCloudScreened
Weight
[[ 5.9143289e+13  6.0605784e+13  6.2138962e+13 ...  5.5159447e+13
   5.6222850e+13  5.7611186e+13]
 [ 5.5106528e+13  5.4806706e+13  5.4454108e+13 ...  5.6082852e+13
   5.5726769e+13  5.5403140e+13]
 [ 5.3136773e+13  5.0562465e+13  5.1282753e+13 ...  5.3069518e+13
   5.3461811e+13  5.3712920e+13]
 ...
 [-1.2676506e+30 -1.2676506e+30 -1.2676506e+30 ... -1.2676506e+30
  -1.2676506e+30 -1.2676506e+30]
 [-1.2676506e+30 -1.2676506e+30 -1.2676506e+30 ... -1.2676506e+30
  -1.2676506e+30 -1.2676506e+30]
 [-1.2676506e+30 -1.2676506e+30 -1.2676506e+30 ... -1.2676506e+30
  -1.2676506e+30 -1.2676506e+30]]


In [120]:
# ── 5. write tidy NetCDF ─────────────────────────────────────────────────────
OUTFILE = "OMNO2d_NE_2005_0101_20days.nc"
ne_no2.to_netcdf(OUTFILE, engine="h5netcdf")
print(f"✅  Saved {OUTFILE}  (shape {ne_no2.shape})")

✅  Saved OMNO2d_NE_2005_0101_20days.nc  (shape (21, 14, 32))
